In [1]:
import pandas as pd

In [2]:
train_dataset = pd.read_csv("train.csv")
eval_dataset = pd.read_csv("evaluation.csv")

In [3]:
import tensorflow as tf
import tensorflow.keras.backend as K
import tensorflow.keras.layers as tfl
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import Input

try:
    import tensorflow_hub as hub
except ModuleNotFoundError:
    %pip install tensorflow_hub
    import tensorflow_hub as hub
    
try:
    from keras_self_attention import SeqSelfAttention
except ModuleNotFoundError:
    %pip install keras-self-attention
    from keras_self_attention import SeqSelfAttention

import numpy as np


# %pip install -q transformers
# %pip install -q -U tensorflow-text
# %pip install -q tf-models-official==2.7.0

import math as m

^C
Note: you may need to restart the kernel to use updated packages.


ModuleNotFoundError: No module named 'keras_self_attention'

In [ ]:
huburl = "https://tfhub.dev/google/universal-sentence-encoder/4" 
fine_tuned_module_object = hub.load(huburl)

In [5]:
def gen_random_batch(in_groups, batch_halfsize = 8):
    text_batch, reason_batch, y_hat = [], [], []
    all_groups = list(range(in_groups[0].shape[0]))
    for match_group in [True, False]:
        group_idx = np.random.choice(all_groups, size = batch_halfsize)
        text_batch += [in_groups[0][c_idx] for c_idx in group_idx]
        if match_group:
            b_group_idx = group_idx
            y_hat += [1]*batch_halfsize
        else:
            # anything but the same group
            non_group_idx = [np.random.choice([i for i in all_groups if i!=c_idx]) for c_idx in group_idx] 
            b_group_idx = non_group_idx
            y_hat += [0]*batch_halfsize
            
        reason_batch += [in_groups[1][c_idx] for c_idx in b_group_idx]
            
    return np.stack(text_batch, 0), np.stack(reason_batch, 0), np.stack(y_hat, 0)

def siam_gen(in_groups, batch_size = 32):
    while True:
        text_stack, reason_stack, y_hatstack = gen_random_batch(in_groups, batch_size//2)
        yield [text_stack, reason_stack], y_hatstack

def euclidean_distance(vectors):
    (featsA, featsB) = vectors
    sumSquared = K.sum(K.square(featsA - featsB), axis=1, keepdims=True)
    return K.sqrt(K.maximum(sumSquared, K.epsilon()))

def contrastive_loss(y, preds, margin=1):
    # explicitly cast the true class label data type to the predicted
    # class label data type (otherwise we run the risk of having two
    # separate data types, causing TensorFlow to error out)
    y = tf.cast(y, preds.dtype)
    # calculate the contrastive loss between the true labels and
    # the predicted labels
    squaredPreds = K.square(preds)
    squaredMargin = K.square(K.maximum(margin - preds, 0))
    loss = K.mean(y * squaredPreds + (1 - y) * squaredMargin)
    return loss

In [6]:
def accuracy(y_true, y_pred):
    results = (y_pred <= 0.5).astype(int).squeeze()
    return np.mean([y_true == results])

In [1]:
tfhub_handle_preprocess = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
tfhub_handle_encoder = "https://tfhub.dev/tensorflow/small_bert/bert_en_uncased_L-4_H-512_A-8/2"
preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess)

In [2]:
def encoder(name):
    inputs = Input(shape=(), dtype=tf.string)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=False, name=name)
    
    encoder_inputs = preprocessing_layer(inputs)
    outputs = encoder(encoder_inputs)
    net = outputs['pooled_output']
    
    a = tfl.Dense(64, activation="linear")(net)
    a = tfl.BatchNormalization()(a)
    a = tfl.Activation('relu')(a)
    a = tfl.Dense(32, activation="linear")(a)
    a = tfl.BatchNormalization()(a)
    outputs = tfl.Activation('relu')(a)
    
    model = Model(inputs, outputs)
    return model

In [4]:
def get_model():
    #textEncoder
    inputText = Input(shape=(), dtype=tf.string)
    text_embedd = encoder("textBertEncoder")(inputText)

    #reasonEncoder
    inputReason = Input(shape=(), dtype=tf.string)
    reason_embedd = encoder("reasonBertEncoder")(inputReason)

    combined_features = tfl.concatenate([text_embedd, reason_embedd], name = 'merge_features')
    print(combined_features.shape)
    combined_features = tfl.Dense(16, activation = 'linear')(combined_features)
    combined_features = tfl.BatchNormalization()(combined_features)
    combined_features = tfl.Activation('relu')(combined_features)
    combined_features = tfl.Dense(4, activation = 'linear')(combined_features)
    combined_features = tfl.BatchNormalization()(combined_features)
    combined_features = tfl.Activation('relu')(combined_features)
    combined_features = tfl.Dense(1, activation = 'sigmoid')(combined_features)
    
    model = Model(inputs = [inputText, inputReason], outputs = [combined_features], name="bertModel")
    return model

In [5]:
model = get_model()

NameError: name 'tf' is not defined

In [ ]:
model.summary()

In [ ]:
model.compile(optimizer="adam", loss = "binary_crossentropy", metrics = [adfsccuracy()])

In [ ]:
loss_history = model.fit(siam_gen([df["clean_text"], df["reason"]], 64), 
                         validation_data = siam_gen([df_test["clean_text"], df_test["reason"]], 32),
                         steps_per_epoch=50, validation_steps=10, epochs = 100, 
                         verbose = True, use_multiprocessing=True)